In [1]:
import scanpy as sc
import anndata as an
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import torch
from sys import getsizeof
import json 

import os, sys
sys.path.append('../src')
from models.lib.data import *
from models.lib.neural import *
from models.lib.lightning_train import DataModule, generate_trainer

datafiles=['../data/mouse/MouseAdultInhibitoryNeurons.h5ad']
labelfiles=['../data/mouse/Adult Inhibitory Neurons in Mouse_labels.tsv']
le = LabelEncoder()

/Users/julian/miniconda3/envs/single-cell/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mo_data = an.read_h5ad('../data/mouse/Mo_PV_paper_TDTomato_mouseonly.h5ad')

/Users/julian/miniconda3/envs/single-cell/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [23]:
sum(mo_data.X.sum(axis=0) == 0)

matrix([[0, 0, 0, ..., 1, 0, 0]])

In [38]:
len([x for x in mo_data.var.index if x.lower().startswith('mrp')])

78

In [39]:
atlas_data = an.read_h5ad('../data/mouse/MouseAdultInhibitoryNeurons.h5ad')

In [40]:
data = AnnDatasetMatrix(
    matrix=mo_data.X,
    labels=mo_data.obs.index,
)

In [76]:
labels = pd.read_csv('../data/mouse/MouseAdultInhibitoryNeurons_labels.csv')
labels['class'].unique()

array(['S-phase_MCM4/H43C', 'Ctx_LHX6/SST', 'Str_LHX8/CHAT', 'Excitatory',
       'Amy/Hypo_HAP1/PEG10', 'GP_GBX1/GABRA1', 'Ctx_PROX1/SNCG',
       'Ctx_PVALB/VIPR2', 'Glia', 'vStr_DRD1/NPY1R', 'Ctx_LHX6/PVALB',
       'Str-IN_CRABP1/MAF', 'Ctx_LAMP5/NDNF', 'MGE_LHX6/NPY',
       'Ctx/BN_SST/CHODL', 'Ctx_PROX1/LAMP5', 'Ctx_CCK/VIP',
       'G2-M_UBE2C/ASPM', 'CGE_NR2F2/PROX1', 'Str-dSPN_FOXP1/ISL1',
       'Str-iSPN_FOXP1/PENK', 'BN-eSPN_FOXP2/TSHZ1', 'LGE_FOXP1/PENK',
       'LGE_FOXP1/ISL1', 'Transition', 'Ctx_CCK/DPY19L1',
       'Ctx_LHX6/LAMP5', 'Ctx_SST/NDNF', 'vSTR_HAP1/ZIC1', 'MGE_LHX6/MAF',
       'VMF_PEG10/DLK1', 'VMF_NR2F2/LHX6', 'OB-PGC_TH/SCGN',
       'OB-GC_STXBP6/PENK', 'LGE_FOXP2/TSHZ1', 'LGE_MEIS2/PAX6/SCGN',
       'OB-PGC_ZIC', 'LGE-OB_MEIS2/PAX6', 'LGE_MEIS2/PAX6', 'OB-GC_RPRM',
       'OB-GC NR2F2/PENK', 'VMF_ZIC1/ZIC2', 'OB-PGC_FOXP2/CALB1',
       'VMF_CRABP1/LHX8', 'VMF_LHX1/POU6F2', 'LGE_FOXP1/ISL1/NPY1R',
       'MGE_CRABP1/MAF', 'Ctx_NR2F2/PAX6', 'RMTW_ZIC1

In [78]:
pd.read_csv('../data/developing_mouse_brain/labels_developing_mouse_brain.tsv', sep='\t')

,CellID,Age,Batch,CellCycle,CellCycle_G1,CellCycle_G2M,CellCycle_S,Cell_Conc,ChipID,Class,...,Punchcard,PunchcardClusters,Region,Replicate,SampleID,SampleName,Sample_Index,Tissue,TotalUMI,ngperul_cDNA
0,10X66_4_A_1:CATTTCGAGATACCx,e16.0,4,0.001595,0.000319,0.001276,0.0,600.0,10X66,Neuron,...,HindbrainE1618_Filtered_Neuronal_Glut,0,Hindbrain,0,10X66_4,G97,SI-3A-C4,Hindbrain,18809.0,"7,7"
1,10X66_4_A_1:ACGTTGGATTTGGGx,e16.0,4,0.001199,0.000479,0.000719,0.0,600.0,10X66,Neuron,...,HindbrainE1618_Filtered_Neuronal_Glut,0,Hindbrain,0,10X66_4,G97,SI-3A-C4,Hindbrain,16686.0,"7,7"
2,10X66_4_A_1:TATGTCTGGTAAAGx,e16.0,4,0.001124,0.000326,0.000798,0.0,600.0,10X66,Neuron,...,HindbrainE1618_Filtered_Neuronal_Glut,0,Hindbrain,0,10X66_4,G97,SI-3A-C4,Hindbrain,27568.0,"7,7"
3,10X68_6_A_1:CCTGACTGAGTCACx,e17.0,5,0.001613,0.000346,0.001267,0.0,600.0,10X68,Neuron,...,HindbrainE1618_Filtered_Neuronal_Glut,0,Hindbrain,0,10X68_6,G101,SI-3A-G4,Hindbrain,8680.0,"7,7"
4,10X68_6_A_1:AGTATAACGAACCTx,e17.0,5,0.001600,0.000200,0.001400,0.0,600.0,10X68,Neuron,...,HindbrainE1618_Filtered_Neuronal_Glut,0,Hindbrain,0,10X68_6,G101,SI-3A-G4,Hindbrain,5001.0,"7,7"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292490,10X70_3_A_1:GTCGAATGGGTGTTx,e17.0,4,0.000772,0.000257,0.000514,0.0,600.0,10X70,Choroid plexus,...,ForebrainVentrothalamicE1618_FilteredOut,1,Forebrain,0,10X70_3,G103,SI-3A-A5,ForebrainVentroThalamic,15553.0,"4,8"
292491,10X70_3_A_1:AGGTCATGCAAGCTx,e17.0,4,0.000998,0.000333,0.000666,0.0,600.0,10X70,Choroid plexus,...,ForebrainVentrothalamicE1618_FilteredOut,1,Forebrain,0,10X70_3,G103,SI-3A-A5,ForebrainVentroThalamic,6010.0,"4,8"
292492,10X70_6_A_1:GCACAATGGATGAAx,e17.0,4,0.001384,0.000260,0.001125,0.0,600.0,10X70,Choroid plexus,...,ForebrainVentrothalamicE1618_FilteredOut,1,Forebrain,1,10X70_6,G106,SI-3A-D5,ForebrainVentroThalamic,11558.0,"9,6"
292493,10X70_3_A_1:CATCAGGAGTGAGGx,e17.0,4,0.003384,0.000785,0.002599,0.0,600.0,10X70,Choroid plexus,...,ForebrainVentrothalamicE1618_FilteredOut,1,Forebrain,0,10X70_3,G103,SI-3A-A5,ForebrainVentroThalamic,20393.0,"4,8"


In [53]:
g1 = [x.lower() for x in atlas_data.var.index]
g2 = [x.lower() for x in mo_data.var.index]

In [79]:
diff = list(set(g1).intersection(set(g2)))

In [80]:
len(diff)

25163

In [82]:
[x for x in diff if x.startswith('malat')]

['malat1']

In [73]:
sum([x.startswith('gm') for x in diff])

7806

In [4]:
# atlas_labels = pd.read_csv('../data/mouse/MouseAdultInhibitoryNeurons_labels.csv')
# le = LabelEncoder()
# atlas_labels['numeric_class'] = le.fit_transform(atlas_labels['class'])

# subset_labels = atlas_labels[atlas_labels['class'].isin(atlas_labels['class'].value_counts()[0:10].index)]
# # subset_labels = subset_labels.iloc[0: 1000, :]
# subset_labels.index.name = 'cell'

# # subset_labels.to_csv('../data/mouse/subset_labels.csv')
# # pd.read_csv('../data/mouse/subset_labels.csv')
# subset_labels = subset_labels.sample(n=1000, weights='numeric_class', random_state=1)
# subset_labels.to_csv('../data/mouse/subset_labels.csv')
# pd.read_csv('../data/mouse/subset_labels.csv')['numeric_class'].value_counts()

In [5]:
# atlas_labels.to_csv('../data/mouse/MouseAdultInhibitoryNeurons_labels.csv', index=False)

# pd.read_csv('../data/mouse/MouseAdultInhibitoryNeurons_labels.csv')

In [6]:
# mouse_atlas = sc.read_h5ad('../data/mouse/MouseAdultInhibitoryNeurons.h5ad')
# mo_data = an.read_h5ad('../data/mouse/Mo_PV_paper_TDTomato_mouseonly.h5ad')

# g1 = mo_data.var.index.values 
# g2 = mouse_atlas.var.index.values

# refgenes = sorted(list(set(g1).intersection(g2)))

In [7]:
# g1, g2, refgenes

## Reading in model from checkpoint

In [11]:
model = TabNetLightning.load_from_checkpoint(
    '../checkpoints/checkpoint-280-desc-mouse.ckpt',
    input_dim=34430,
    output_dim=50,
)

Initializing network
Initializing explain matrix


In [ ]:
from os.path import join
data_path = '../data/mouse//'
datafiles=[join(data_path, 'MouseAdultInhibitoryNeurons.h5ad')]
labelfiles=[join(data_path, 'MouseAdultInhibitoryNeurons_labels.csv')]

device = ('cuda:0' if torch.cuda.is_available() else None)

module = DataModule(
    datafiles=datafiles,
    labelfiles=labelfiles,
    class_label='numeric_class',
    batch_size=8,
    num_workers=0,
    shuffle=True,
    drop_last=True,
    normalize=True,
)

module.setup()

In [ ]:
module(next(iter(module.trainloader)))

In [ ]:
trainer = pl.Trainer()

trainer.test()

# Mouse model training with generate_trainer test

In [8]:
# Define labelfiles and trainer 
from os.path import join
data_path = '../data/mouse//'
datafiles=[join(data_path, 'MouseAdultInhibitoryNeurons.h5ad')]
labelfiles=[join(data_path, 'MouseAdultInhibitoryNeurons_labels.csv')]

device = ('cuda:0' if torch.cuda.is_available() else None)

module = DataModule(
    datafiles=datafiles,
    labelfiles=labelfiles,
    class_label='numeric_class',
    batch_size=8,
    num_workers=32,
    shuffle=True,
    drop_last=True,
    normalize=True,
)

In [9]:
module.setup()

Creating train/val/test DataLoaders...


KeyboardInterrupt: 

## Code for slides

This doesn't actually compile, lol

In [ ]:
trainer, model, module = generate_trainer(
    datafiles=['dataset_inhibitory.h5ad', 'dataset_excitatory.csv', 'glial_cells.h5ad'],
    labelfiles=['inhibitory_labels.csv', 'excitatory_labels.csv', 'glial_labels.csv'],
    class_label='cell_type',
)

trainer.fit(model, datamodule=module)

In [10]:
trainer.fit(model, datamodule=module)

Creating train/val/test DataLoaders...
Done, continuing to training.


wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name       | Type                 | Params
----------------------------------------------------
0 | base_model | TabNetGeneClassifier | 2.3 M 
----------------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.113     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:453: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of 

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


# Mouse Model training code using lib.lightning_data

In [22]:
labelfile = '../data/mouse/MouseAdultInhibitoryNeurons_labels.csv'
label_df = pd.read_csv(labelfile)


label_df

,class,numeric_class
0,S-phase_MCM4/H43C,36
1,S-phase_MCM4/H43C,36
2,Ctx_LHX6/SST,9
3,Str_LHX8/CHAT,40
4,Str_LHX8/CHAT,40
...,...,...
141064,S-phase_MCM4/H43C,36
141065,Transition,41
141066,Transition,41
141067,S-phase_MCM4/H43C,36


In [ ]:
test_prop=0.2
# trainsplit, valsplit = train_test_split(current_labels, stratify=current_labels, test_size=test_prop)
# trainsplit, testsplit = train_test_split(trainsplit, stratify=trainsplit, test_size=test_prop)


# trainsplit.index

In [23]:
module = DataModule(
    datafiles=['../data/mouse/MouseAdultInhibitoryNeurons.h5ad'],
    labelfiles=['../data/mouse/MouseAdultInhibitoryNeurons_labels.csv'],
    class_label='numeric_class',
    collocate=False,
    batch_size=4,
    num_workers=0,
    refgenes=refgenes,
    currgenes=g2,
)

module

In [ ]:
module.setup()

Creating train/val/test DataLoaders...


/Users/julian/Documents/Projects/organoid-classification/notebooks/../src/models/lib/data.py:601: UserWarning: collocate =False, so multiple files will return multiple DataLoaders and cannot be trained sequentially with PyTorch-Lightning
  warnings.warn(f"{collocate =}, so multiple files will return multiple DataLoaders and cannot be trained sequentially with PyTorch-Lightning")


In [ ]:
train = module.train_dataloader()

print(next(iter(train)))

In [ ]:
train = DataLoader(atlas_train, batch_size=4, num_workers=0, drop_last=True, shuffle=True)
val = DataLoader(atlas_val, batch_size=4, num_workers=0, drop_last=True, shuffle=False)

In [ ]:
# from models.lib.neural import GeneClassifier
# from pytorch_lightning import Trainer 
# from pytorch_lightning.loggers import WandbLogger

# model = GeneClassifier(
#     input_dim=34430, 
#     output_dim=50,
#     optim_params={
#         'optimizer': torch.optim.Adam,
#         'lr': 3e-4,
#     }
# )

# wandb_logger = WandbLogger(project='Mouse Classifier', name='Tabnet with Metrics')
# trainer = Trainer(logger=wandb_logger)

# trainer.fit(model, train, val)

In [ ]:
def gene_intersection(
    files
):
    import dask.dataframe as dd 
    
    cols = []
    for file in files:
        temp = pd.read_csv(fpath, nrows=1, header=1).columns 
        temp = [x.split('|')[0].upper().strip() for x in temp]
        cols.append(set(temp))
    
    unique = list(set.intersection(*cols))
    unique = sorted(unique)
    
    return unique 

In [ ]:
generate_single_dataset(
    datafile='../data/mouse/MouseAdultInhibitoryNeurons.h5ad',
    labelfile='../data/mouse/Adult Inhibitory Neurons in Mouse_labels.tsv',
    class_label='numeric_class',
    sep='\t'
)